In [1]:
import finnhub
import pandas as pd
import numpy as np
import pyodbc

finnhub_client = finnhub.Client(api_key="")

tickers = ["AMD", "NVDA", "AMZN", "PLTR", "TSLA", "AAPL"]
profile = []

for t in tickers :
    data = finnhub_client.company_profile2(symbol=t)
    if data :
        profile.append(data)

df = pd.DataFrame(profile)

In [2]:
df = df[['ticker', 'name', 'country', 'finnhubIndustry', 'exchange', 'currency']]
df.rename(columns = 
    {'name' : 'company_name', 
     'finnhubIndustry' : 'industry', 
     'exchange' : 'market'},
    inplace = True)
df

,ticker,company_name,country,industry,market,currency
0,AMD,Advanced Micro Devices Inc,US,Semiconductors,NASDAQ NMS - GLOBAL MARKET,USD
1,NVDA,NVIDIA Corp,US,Semiconductors,NASDAQ NMS - GLOBAL MARKET,USD
2,AMZN,Amazon.com Inc,US,Retail,NASDAQ NMS - GLOBAL MARKET,USD
3,PLTR,Palantir Technologies Inc,US,Technology,NASDAQ NMS - GLOBAL MARKET,USD
4,TSLA,Tesla Inc,US,Automobiles,NASDAQ NMS - GLOBAL MARKET,USD
5,AAPL,Apple Inc,US,Technology,NASDAQ NMS - GLOBAL MARKET,USD


In [3]:
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=Kiyoworld;"
    "DATABASE=USStocks;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

In [4]:
try :
    insert_query = "INSERT INTO clean.dim_ticker (ticker, company_name, country, industry, market, currency) VALUES (?, ?, ?, ?, ?, ?)"
    
    print("Loading US stock ticker data into SQL Server...")
    
    rows = list(df.itertuples(index=False, name=None))
    cursor.fast_executemany = True
    cursor.executemany(insert_query, rows)
    conn.commit()
    print(f"Successfully inserted {len(df)} records.") 

except Exception as e :
    print(f"Error : {e}")

finally:
    cursor.close()
    conn.close()

Loading US stock ticker data into SQL Server...
Successfully inserted 6 records.
